In [59]:
# carregar bibliotecas
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.stattools as ts
import statsmodels.regression.linear_model as rg
import arch.unitroot as at
from statsmodels.tsa.stattools import adfuller


In [60]:
#path = 'C:/Users/vinicius_coin/Desktop/DADOS_MESTRADO/bovesparser-master/data/RESULTSET/'
path = 'C:/Users/vinicius_coin/Desktop/DADOS_MESTRADO/bovesparser-master/data/'
dfbovespa = pd.read_csv(path+"PREBASE/COTACOES_consolidado.csv", sep = ';',encoding="ISO-8859-1")
ibovespa = pd.read_csv(path+"/APOIO/ibovespa.csv", sep = ',',encoding="ISO-8859-1",usecols = ["Date","Close"])
ibovespa["Date"] = pd.to_datetime(ibovespa["Date"])
lista_papeis = dfbovespa.cod_papel.unique()

In [61]:
#Pega a data da contacao da petrobras e torna index
df_comp = pd.DataFrame()
df_acao = dfbovespa[dfbovespa["cod_papel"] == 'PETR3']
df_comp['data_pregao'] =  pd.to_datetime(df_acao['data_pregao'].values)
len(df_acao)

2472

In [62]:
for papel in lista_papeis:
    if papel[-1] in ['3', '4']:
        df_acao = dfbovespa[dfbovespa["cod_papel"] == papel]
        if len(df_acao) == 2472:
            df_comp[papel] = df_acao['preco_ultimo'].values
df_comp.set_index('data_pregao', inplace=True)
df_comp = df_comp['2017-06-30':'2019-06-30']
df_comp.head()

,ABCB4,ALPA4,BBAS3,BBDC3,BBDC4,BBRK3,BEEF3,BRAP4,BRFS3,BRML3,...,SBSP3,SLCE3,SMTO3,TCSA3,TGMA3,TOTS3,TRPL4,USIM3,VALE3,WEGE3
data_pregao,,,,,,,,,,,,,,,,,,,,,
2017-06-30,16.79,13.71,26.80,27.60,28.15,1.16,12.31,20.60,39.20,11.95,...,31.66,21.10,17.20,2.05,13.80,30.16,64.50,8.83,29.00,17.70
2017-07-03,16.10,13.90,27.17,27.70,28.20,1.16,12.39,21.09,39.37,12.01,...,31.87,21.20,17.16,2.08,13.80,30.02,64.35,9.00,29.65,17.95
2017-07-04,15.88,13.91,27.31,27.66,28.22,1.16,12.45,21.18,39.16,12.03,...,31.85,21.20,17.28,2.11,13.84,30.46,64.30,9.05,29.62,18.02
2017-07-05,15.82,13.75,27.17,27.72,28.23,1.16,12.34,20.76,39.08,12.05,...,31.90,21.29,16.78,2.11,13.79,30.06,64.60,9.21,28.93,18.11
2017-07-06,15.85,13.75,26.99,27.17,27.82,1.15,12.56,20.95,37.86,11.74,...,31.70,20.50,16.79,2.10,13.76,29.74,66.55,9.60,28.92,17.72


In [63]:
#from statsmodels.tsa.stattools import coint
dfcoint = pd.DataFrame(columns=['papel1', 'papel2', 'coint'])
lista_papeis = list(df_comp)
from statsmodels.tsa.stattools import adfuller
previous_nested_paper = list()

x = 0
for i in lista_papeis:
    previous_nested_paper.append(i)
    for j in lista_papeis:
        if j not in previous_nested_paper:
            if(i != j):
                   
                score, pvalue, _ = ts.coint(df_comp[i], df_comp[j])
                if pvalue <= 0.05:
                    train_spread = df_comp[i] - df_comp[j]
                    result = adfuller(train_spread)
                    if(result[1] <= 0.05):
                        dfcoint.loc[x] = [i,j,pvalue]
                x = x+1
dfcoint.sort_values(by='coint', ascending=True).head(50)

,papel1,papel2,coint
1402,ELET3,LREN3,0.000194
110,ALPA4,IGTA3,0.000360
1023,CMIG4,LOGN3,0.001830
514,BRAP4,GGBR4,0.002721
308,BBDC4,EZTC3,0.003438
164,BBAS3,ELET3,0.006167


In [64]:
len(dfcoint)

6

In [65]:
dfcoint.sort_values(by='coint', ascending=True).head(20)

,papel1,papel2,coint
1402,ELET3,LREN3,0.000194
110,ALPA4,IGTA3,0.000360
1023,CMIG4,LOGN3,0.001830
514,BRAP4,GGBR4,0.002721
308,BBDC4,EZTC3,0.003438
164,BBAS3,ELET3,0.006167
